# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [41]:
import numpy as np
import os
# from scipy.misc import imread, imresize
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [42]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf

tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [43]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 256

#batch_size = 8 -> overfit -> memory efficient -> slow training
#batch_sie= 32/16 
#batch_size = 512 -> underfit -> need more memory -> fast training

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [44]:
img = cv2.imread("Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png")

In [45]:
import matplotlib.pyplot as plt

In [46]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [47]:
r_img = cv2.resize(img, (100,100))

In [48]:
plt.imshow(cv2.cvtColor(r_img, cv2.COLOR_BGR2RGB))

In [49]:
r_img[:,:,0]

array([[106, 113, 108, ..., 127, 121, 120],
       [111, 116, 114, ..., 127, 120, 118],
       [117, 126, 124, ..., 121, 117, 119],
       ...,
       [104,  58,  38, ...,  49,  51,  53],
       [ 57,  35,  26, ...,  47,  49,  52],
       [ 16,  15,  18, ...,  45,  50,  53]], dtype=uint8)

In [50]:
r_img.shape

(100, 100, 3)

In [51]:
#r_img

In [52]:
c_img = r_img[10:90, 10:90, :]

In [53]:
plt.imshow(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB))

In [54]:
c_img.shape

(80, 80, 3)

In [55]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [3,6,9,12,15,18,21] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,len(img_idx),80,80,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # crop & resize to 60x60
                    
                    resized_image = cv2.resize(img, (100,100))
                    cropped_image = resized_image[10:90, 10:90, :]
                    
                    batch_data[folder,idx,:,:,0] = cropped_image[:,:,0]/float(255)
                    batch_data[folder,idx,:,:,1] = cropped_image[:,:,1]/float(255)
                    batch_data[folder,idx,:,:,2] = cropped_image[:,:,2]/float(255)
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if len(folder_list)/batch_size != 0:
            last_batch_size = len(foler_list)%batch_size
            batch_data = np.zeros((last_batch_size,len(img_idx),80,80,3))
            batch_labels = np.zeros((last_batch_size,5))
            batch_index = num_batches+1
            for folder in range(last_batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch_index*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (batch_index*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # crop & resize to 60x60
                    
                    resized_image = cv2.resize(img, (100,100))
                    cropped_image = resized_image[10:90, 10:90, :]
                    
                    batch_data[folder,idx,:,:,0] = cropped_image[:,:,0]/float(255)
                    batch_data[folder,idx,:,:,1] = cropped_image[:,:,1]/float(255)
                    batch_data[folder,idx,:,:,2] = cropped_image[:,:,2]/float(255)
                    
                batch_labels[folder, int(t[folder + (batch_index*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
            
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [56]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 1
print ('# epochs =', num_epochs)
num_classes=5
print('# num classes=', num_classes)

# training sequences = 663
# validation sequences = 100
# epochs = 1
# num classes= 5


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [57]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

model = Sequential()
model.add(Conv3D(32, (3, 3, 3), padding='same', input_shape=(7, 80, 80, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [58]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_6 (Conv3D)           (None, 7, 80, 80, 32)     2624      
                                                                 
 activation_12 (Activation)  (None, 7, 80, 80, 32)     0         
                                                                 
 batch_normalization_6 (Batc  (None, 7, 80, 80, 32)    128       
 hNormalization)                                                 
                                                                 
 conv3d_7 (Conv3D)           (None, 5, 78, 78, 64)     55360     
                                                                 
 activation_13 (Activation)  (None, 5, 78, 78, 64)     0         
                                                                 
 batch_normalization_7 (Batc  (None, 5, 78, 78, 64)    256       
 hNormalization)                                      

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [59]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [60]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [61]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [62]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 256


<class 'KeyError'>: 'Failed to format this callback filepath: "model_init_2021-12-2200_13_22.553701/model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5". Reason: \'val_loss\''